In [49]:
# Import libraries 
import pandas as pd
import numpy as np
import warnings #Import warnings module to handle warnings
warnings.filterwarnings('ignore', category=DeprecationWarning) #Filter DeprecationWarning to ignore the warnings
import matplotlib.pyplot as plt 
from keras.models import Model ,Sequential 
from keras.layers import Embedding ,Dense ,Dropout , LSTM ,Input ,Activation , Bidirectional ,GlobalMaxPool1D #Import various types of layers from keras.layers module
from tensorflow.keras.preprocessing.sequence import pad_sequences #Import pad_sequences function from tensorflow.keras.preprocessing.sequence module
from keras.initializers import glorot_uniform
%matplotlib inline

""" The input data needs to be preprocessed using the `pad_sequences` function.
Also, the code is using the `glorot_uniform` initializer to initialize the weights of the neural network.
Finally, the code sets up the environment to display the plots inline using %matplotlib inline."""

' The input data needs to be preprocessed using the `pad_sequences` function.\nAlso, the code is using the `glorot_uniform` initializer to initialize the weights of the neural network.\nFinally, the code sets up the environment to display the plots inline using %matplotlib inline.'

In [2]:
# Load the data 
neg_train=pd.read_csv('train_Arabic_tweets_negative_20190413.tsv',sep='\t')
neg_train.columns=['sentiment','text']
len(neg_train)
pos_train=pd.read_csv('train_Arabic_tweets_positive_20190413.tsv',sep='\t')
pos_train.columns=['sentiment','text']
print(len(pos_train))
print(len(neg_train))

22760
22513


In [3]:
# Concatenate data
data = pd.concat([neg_train, pos_train])
len(data)

45273

In [52]:
## shufle data
from sklearn.utils import shuffle
shuffled_data = shuffle(data)
shuffled_data.head()

,sentiment,text
7997,pos,📹 الرابر جيني كيم في كوتشيلا 🔥 - ابريل ،
1656,pos,✍ قال الإمام ابن القيم رحمه الله : من اعتاد ⁧ ...
21375,neg,ٲليس من العيب ان تحفظ كل الأغاني .. وتكرر نفس ...
13848,pos,:40 الوضع امان الحمدلله .. ماف انباء عن اصابات...
9552,pos,ومن أحياها فكأنما أحيا الناس جميعا طفلة عمرها ...


In [53]:
## convert pos to 1 and neg to 0
shuffled_data['sentiment']=shuffled_data['sentiment'].apply(lambda x: 1 if x =='pos' else 0)

In [54]:
shuffled_data.head()

,sentiment,text
7997,1,📹 الرابر جيني كيم في كوتشيلا 🔥 - ابريل ،
1656,1,✍ قال الإمام ابن القيم رحمه الله : من اعتاد ⁧ ...
21375,0,ٲليس من العيب ان تحفظ كل الأغاني .. وتكرر نفس ...
13848,1,:40 الوضع امان الحمدلله .. ماف انباء عن اصابات...
9552,1,ومن أحياها فكأنما أحيا الناس جميعا طفلة عمرها ...


In [7]:
# pre-trained Word2Vec model
from gensim.models import KeyedVectors
def load_w2v(file_path,binary):
    return KeyedVectors.load_word2vec_format(file_path,binary=binary)

In [8]:
w2v = load_w2v("wiki.ar.vec", binary=False)

In [10]:
print(len(w2v.key_to_index))

610977


In [12]:
max_sequence_length = 85
max_num_words = len(w2v.index_to_key)
embedding_dim = 300


In [13]:
sample=w2v['حسن']
print(sample.shape)
print(w2v.most_similar('حسن'))

(300,)
[('،حسن', 0.6795670390129089), ('أبوحسن', 0.6792263388633728), ('#حسن', 0.6771580576896667), ('وحسن', 0.6517007350921631), ('لحسن', 0.6505534648895264), ('\u200fحسن', 0.6446623206138611), ('كحسن', 0.6202388405799866), ('عبدالحسن', 0.6161733865737915), ('بوحسن', 0.6157496571540833), ('د/حسن', 0.6062546372413635)]


In [21]:
import os 
import glob
import codecs
from keras.preprocessing.text import Tokenizer ,text_to_word_sequence
def get_stop_words():
    path='list.txt'
    stop_words = []
    with codecs.open(path, "r", encoding="utf-8", errors="ignore") as myfile:
        stop_words = myfile.readlines()
    stop_words = [word.strip() for word in stop_words]
    return stop_words
stop_words=get_stop_words()
print(stop_words)

['،', 'ء', 'ءَ', 'آ', 'آب', 'آذار', 'آض', 'آل', 'آمينَ', 'آناء', 'آنفا', 'آه', 'آهاً', 'آهٍ', 'آهِ', 'أ', 'أبدا', 'أبريل', 'أبو', 'أبٌ', 'أجل', 'أجمع', 'أحد', 'أخبر', 'أخذ', 'أخو', 'أخٌ', 'أربع', 'أربعاء', 'أربعة', 'أربعمئة', 'أربعمائة', 'أرى', 'أسكن', 'أصبح', 'أصلا', 'أضحى', 'أطعم', 'أعطى', 'أعلم', 'أغسطس', 'أفريل', 'أفعل به', 'أفٍّ', 'أقبل', 'أكتوبر', 'أل', 'ألا', 'ألف', 'ألفى', 'أم', 'أما', 'أمام', 'أمامك', 'أمامكَ', 'أمد', 'أمس', 'أمسى', 'أمّا', 'أن', 'أنا', 'أنبأ', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'أنتِ', 'أنشأ', 'أنه', 'أنًّ', 'أنّى', 'أهلا', 'أو', 'أوت', 'أوشك', 'أول', 'أولئك', 'أولاء', 'أولالك', 'أوّهْ', 'أى', 'أي', 'أيا', 'أيار', 'أيضا', 'أيلول', 'أين', 'أيّ', 'أيّان', 'أُفٍّ', 'ؤ', 'إحدى', 'إذ', 'إذا', 'إذاً', 'إذما', 'إذن', 'إزاء', 'إلى', 'إلي', 'إليكم', 'إليكما', 'إليكنّ', 'إليكَ', 'إلَيْكَ', 'إلّا', 'إمّا', 'إن', 'إنَّ', 'إى', 'إياك', 'إياكم', 'إياكما', 'إياكن', 'إيانا', 'إياه', 'إياها', 'إياهم', 'إياهما', 'إياهن', 'إياي', 'إيهٍ', 'ئ', 'ا', 'ا?', 'ا?ى', 'االا', 'االتى', 'اب

In [22]:
## remove stop words from text
shuffled_data['text']=shuffled_data['text'].apply(lambda x : [item for item in x.split() if item not in stop_words])

In [23]:
shuffled_data.text

20096    [يخبرونك, :, سار, الدرب, وصل, يخبرك, بالقصة, ا...
6140     [بمناسبة, فوز, الهلال, .., 💙, سحب, آيفون, XR📱,...
777      [الخير, يجماعه،, تقريبا, احلى, نصيحه, انا, شفت...
10959    [الهلال, ينتصر, .., بعيد, مستواه, !, إهدار, غر...
14928    [🎀, نكتب, لنتنفس, حروفنا, ولكي, نتعافى, أوجاعن...
                               ...                        
7607                                         [غرت, ناس, 💔]
13890    [بمناسبة, فوز, الهلال, .., 💙, سحب, آيفون, XR📱,...
2798     [سألوا, حكيما:, لماذا, ترد, يسيئون, لك؟, فرد, ...
14024    [قلبك, يستحق, الفرح, واستودعه, الله, واطمئن, 🌹...
8103        [نرجع, نقول, العالم, مليان, ناس, فيهم, لطف, 💕]
Name: text, Length: 45273, dtype: object

In [24]:
## tokenize data
# It easily vectorize text data by converting words into integer indices, and it also provides several options for handling text preprocessing tasks such as filtering out stop words, lowercasing text, and more.
tokenizer= Tokenizer()
tokenizer.fit_on_texts(shuffled_data['text']) # This step updates the tokenizer's internal vocabulary based on the frequency of the words in the text corpus.
word_index=tokenizer.word_index # This line creates a dictionary of word-to-index mappings based on the updated vocabulary of the tokenizer. 
vocab_size=len(word_index)
print(vocab_size)

83180


In [25]:
train_sequences=tokenizer.texts_to_sequences(shuffled_data['text'])
train_paded_sequences=pad_sequences(train_sequences,maxlen=max_sequence_length,padding='post',truncating='post')

In [26]:
len(train_paded_sequences)

45273

In [27]:
from sklearn.model_selection import train_test_split
train_paded_sequences,valid_paded_sequences,y_train,y_valid=train_test_split(train_paded_sequences,shuffled_data['sentiment'].values,test_size=.2)

In [42]:
embedding_matrix = np.zeros((max_num_words, embedding_dim))
for word, i in word_index.items():
    if word in w2v.key_to_index:
        embedding_matrix[i] = w2v.get_vector(word)

In [46]:
### build model
sentence_indices = Input(shape=(max_sequence_length,),dtype='int32')
embedding_layer = Embedding(vocab_size+1 , embeding_dim,  input_length=max_sequence_length)
embeddings = embedding_layer(sentence_indices)   
X = LSTM(60, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(embeddings)
X = GlobalMaxPool1D()(X)
X = Dropout(0.2)(X)
X = Dense(128)(X)
X = Activation("relu")(X)
X = Dropout(0.2)(X)
X = Dense(512)(X)
X = Activation("relu")(X)
X = Dropout(0.2)(X)
X = Dense(1)(X)
X = Activation('softmax')(X)
model = Model(inputs=sentence_indices,outputs=X)

In [47]:
model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=['accuracy'])

In [48]:
history = model.fit(train_paded_sequences, y_train, batch_size=32, epochs=10, validation_data=(valid_paded_sequences, y_valid))

Epoch 1/10
1132/1132 [==============================] - 419s 363ms/step - loss: 0.2338 - accuracy: 0.5045 - val_loss: 0.1342 - val_accuracy: 0.4956
Epoch 2/10
1132/1132 [==============================] - 386s 341ms/step - loss: 0.0900 - accuracy: 0.5045 - val_loss: 0.1321 - val_accuracy: 0.4956
Epoch 3/10
1132/1132 [==============================] - 377s 333ms/step - loss: 0.0565 - accuracy: 0.5045 - val_loss: 0.1201 - val_accuracy: 0.4956
Epoch 4/10
1132/1132 [==============================] - 382s 337ms/step - loss: 0.0421 - accuracy: 0.5045 - val_loss: 0.1374 - val_accuracy: 0.4956
Epoch 5/10
1132/1132 [==============================] - 392s 347ms/step - loss: 0.0353 - accuracy: 0.5045 - val_loss: 0.1627 - val_accuracy: 0.4956
Epoch 6/10
1132/1132 [==============================] - 387s 342ms/step - loss: 0.0301 - accuracy: 0.5045 - val_loss: 0.1668 - val_accuracy: 0.4956
Epoch 7/10
1132/1132 [==============================] - 380s 336ms/step - loss: 0.0276 - accuracy: 0.5045 - val_